## Transfer Learning using architecture of pre-trained model

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load the architecture of pretrained model
5. Train the model
6. Get predictions

## 1. Import neccessary libraries

In [20]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Model
from tensorflow.keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# unzipping the Dataset.zip file
!unzip /content/drive/My\ Drive/Dataset.zip

Archive:  /content/drive/My Drive/Dataset.zip
replace Dataset/emergency_classification.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Dataset/images/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Dataset/images/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
#reading the csv file containing data labels
data = pd.read_csv('Dataset/emergency_classification.csv')

In [24]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('Dataset/images/' + img_name)
    # pile it one over the other
    X.append(img)
    
# convert this python list to a single numpy array
X = np.array(X)

In [25]:
#getting the labels for images
y = data.emergency_or_not.values

## 3. Pre-Process the data

Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [26]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X)

In [27]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency
# with only one of them is true for every image
y = to_categorical(y)

In [28]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load the architecture of pretrained model

In [29]:
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

In [30]:
#shows model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [31]:
# creating our own model 
x = Dense(100, activation='relu', name='fc1')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='prediction')(x)
my_model = Model(inputs=base_model.input, outputs=y)
my_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [32]:
# compile the model
my_model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

## 5. Train the model

In [33]:
# training the model
my_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
52/52 [==============================] - 25s 457ms/step - loss: nan - accuracy: 0.5784 - val_loss: nan - val_accuracy: 0.5609
Epoch 2/10
52/52 [==============================] - 24s 454ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 3/10
52/52 [==============================] - 24s 460ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 4/10
52/52 [==============================] - 24s 454ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 5/10
52/52 [==============================] - 24s 453ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 6/10
52/52 [==============================] - 24s 454ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 7/10
52/52 [==============================] - 24s 455ms/step - loss: nan - accuracy: 0.5863 - val_loss: nan - val_accuracy: 0.5609
Epoch 8/10
52/52 [=======================

## 6. Get Predictions

In [34]:
# get predictions
predictions = my_model.predict(X_valid)
predicted_class = np.argmax(predictions,axis=1)

In [36]:
# checking the performance of model on validation set
accuracy_score(y_valid[:,1], predicted_class)

0.5609065155807366